<a href="https://colab.research.google.com/github/mimranbee24seecs-ctrl/MLPROJECT/blob/main/GOOGLE%20COLAB%20V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import files

# Upload kaggle.json
if not os.path.exists('/content/kaggle.json'):
    files.upload()

# Configure Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip the Dog Breed dataset
!kaggle competitions download -c dog-breed-identification
!unzip -q dog-breed-identification.zip -d /content/dog_data
print("Data is ready!")

Saving kaggle.json to kaggle.json
 87% 601M/691M [00:03<00:01, 73.6MB/s]
100% 691M/691M [00:03<00:00, 204MB/s] 
Data is ready!


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configuration from your code
IMAGE_SIZE = (331, 331)
batchSize = 8

# Load labels and fix filenames
df = pd.read_csv('/content/dog_data/labels.csv')
df['id'] = df['id'] + '.jpg'

# Create Data Generator (handles normalization from 0-255 to 0-1)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

# Training Set
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='/content/dog_data/train',
    x_col="id",
    y_col="breed",
    subset="training",
    batch_size=batchSize,
    target_size=IMAGE_SIZE,
    class_mode="categorical"
)

# Validation Set (Replaces your train_test_split)
valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='/content/dog_data/train',
    x_col="id",
    y_col="breed",
    subset="validation",
    batch_size=batchSize,
    target_size=IMAGE_SIZE,
    class_mode="categorical"
)

Found 7156 validated image filenames belonging to 120 classes.
Found 3066 validated image filenames belonging to 120 classes.


In [ ]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.optimizers import Adam

IMAGE_FULL_SIZE = (331, 331, 3) #

# Build the model
myModel = NASNetLarge(input_shape=IMAGE_FULL_SIZE, weights='imagenet', include_top=False)

# Freeze layers
for layer in myModel.layers:
    layer.trainable = False

plusFlattenLayer = Flatten()(myModel.output) #

# Add prediction layer for 120 breeds
predicition = Dense(120, activation='softmax')(plusFlattenLayer)

model = Model(inputs=myModel.input, outputs=predicition)

# Compile with your learning rate
lr = 1e-4
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr),
    metrics=['accuracy']
)

343610240/343610240 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Save model to your Google Drive folder
best_model_file = "/content/drive/MyDrive/dogs_model.h5"

# Your callbacks list
callbacks_list = [
    ModelCheckpoint(best_model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor='val_accuracy', patience=7, verbose=1)
]

# Train (fit) using generators
r = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    callbacks=callbacks_list # Removed the double brackets
)

Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


895/895 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - accuracy: 0.7505 - loss: 1.9684
Epoch 1: val_loss improved from inf to 1.82997, saving model to /content/drive/MyDrive/dogs_model.h5


895/895 ━━━━━━━━━━━━━━━━━━━━ 466s 444ms/step - accuracy: 0.7506 - loss: 1.9681 - val_accuracy: 0.8702 - val_loss: 1.8300 - learning_rate: 1.0000e-04
Epoch 2/30
895/895 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.9614 - loss: 0.3920
Epoch 2: val_loss improved from 1.82997 to 1.70591, saving model to /content/drive/MyDrive/dogs_model.h5


895/895 ━━━━━━━━━━━━━━━━━━━━ 288s 321ms/step - accuracy: 0.9614 - loss: 0.3920 - val_accuracy: 0.8969 - val_loss: 1.7059 - learning_rate: 1.0000e-04
Epoch 3/30
895/895 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.9824 - loss: 0.1767
Epoch 3: val_loss did not improve from 1.70591
895/895 ━━━━━━━━━━━━━━━━━━━━ 272s 304ms/step - accuracy: 0.9824 - loss: 0.1767 - val_accuracy: 0.9015 - val_loss: 1.7493 - learning_rate: 1.0000e-04
Epoch 4/30
895/895 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.9896 - loss: 0.1172
Epoch 4: val_loss did not improve from 1.70591
895/895 ━━━━━━━━━━━━━━━━━━━━ 270s 302ms/step - accuracy: 0.9896 - loss: 0.1172 - val_accuracy: 0.9044 - val_loss: 1.8807 - learning_rate: 1.0000e-04
Epoch 5/30
895/895 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.9885 - loss: 0.1404
Epoch 5: val_loss did not improve from 1.70591

Epoch 5: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
895/895 ━━━━━━━━━━━━━━━━━━━━ 271s 302ms/step - accuracy: 0.9885 - loss: